## Markers Calcul 002 (our data)

## Task Parameters and Raw CSV Loading
### Data Preparation: Centering, Radial Distance Computation, and Record Definitions
#### Filtering: Removing Samples Before the First Entry into the Target

In [ ]:
import numpy as np

# Task parameters (fixed values)
cornerX = 435
cornerY = 80
centerX = 723
centerY = 368
externalRadius = 288
internalRadius = 208
borderRadius = 1
cursorRadius = 16
indexOfDifficulty = 33.086986458020164
taskRadius = 247.5
taskTolerance = 47
cycleMaxNumber = 6
cycleDuration = 20


data = np.loadtxt("/Users/matysprecloux/Desktop/Master IEAP/IEAP-project/Circular-task-analysis:/Data/002MoDe_R1.csv",
                  delimiter=',',skiprows=1)

# skiprows=1 -> skip the header containing column names

timestamps_ms = data[:,0]
mouseX = data[:,1]
mouseY = data[:,2]
mouseInTarget = data[:,3]

# Recenter the coordinate system
x_centered = mouseX - centerX
y_centered = mouseY - centerY
dist = np.sqrt(x_centered**2 + y_centered**2)

# Convert to seconds from the start of the file
t0 = timestamps_ms[0]
timestamps = (timestamps_ms - t0) / 1000.0

# Define start and end times of recordings (in seconds)
record_intervals = [
    (0.022, 20.004),
    (40.429, 60.018),
    (80.371, 100.050),
    (120.608, 140.069),
    (160.146, 180.098)
]

# inside = 1 if INSIDE the ring [internalRadius, externalRadius]
is_inside_csv = (dist >= internalRadius) & (dist <= externalRadius)
flags = is_inside_csv.astype(int)
record_times = record_intervals

def extract_record(times, x, y, inTarget, start, end):
    """
    Extracts a segment of the data between start and end times.
    """
    mask = (times >= start) & (times <= end)

    return (
        times[mask],
        x[mask],
        y[mask],
        inTarget[mask]
    )
all_records = []

for (start, end) in record_times:
    rec_t, rec_x, rec_y, rec_in = extract_record(
        timestamps, mouseX, mouseY, mouseInTarget,
        start, end
    )
    all_records.append((rec_t, rec_x, rec_y, rec_in))

# ----- Step 1 : Remove everything before first entry into target -----
first_inside_indices = np.where(flags == 1)[0]
if len(first_inside_indices) > 0:
    first_idx = first_inside_indices[0]
    # keep only the true circular phase
    t = timestamps[first_idx:]
    x = mouseX[first_idx:]
    y = mouseY[first_idx:]
    flags = flags[first_idx:]
    

## Computation of nLaps (Number of Completed Laps)

In [ ]:

for i, (start, end) in enumerate(record_intervals):

    mask = (timestamps >= start) & (timestamps <= end)

    t_all = timestamps[mask]
    x_all = x_centered[mask]
    y_all = y_centered[mask] 
    dist_all = dist[mask]
    inside_all = is_inside_csv[mask]

    if len(t_all) == 0:
        print(f"Rec{i+1:03d} ,   0.00 ,     0.00 ,     0.00 ,    nan ,    nan ,    nan ,    0.00 ,    0.00")
        continue

    
    # STEP 1 — keep only data from first entry into the target
    
    first_inside_idx = np.where(inside_all)[0]
    if len(first_inside_idx) > 0:
        fi = first_inside_idx[0]
        t_all = t_all[fi:]
        x_all = x_all[fi:]
        y_all = y_all[fi:]
        dist_all = dist_all[fi:]
        inside_all = inside_all[fi:]


    # STEP 2 - nLaps with first inside idx 
def compute_radius_angle(x, y, centerX, centerY):
    """
    Compute radial distance and angle relative to the task center.

    Parameters
    x, y : array-like
        Cursor coordinates.
    centerX, centerY : float
        Target center coordinates.

    Returns
    r : ndarray
        Radial distance from center.
    theta : ndarray
        Angular position (radians, wrapped in [-π, π]).
    """
    r = np.sqrt((x - centerX)**2 + (y - centerY)**2)
    theta = np.arctan2(y - centerY, x - centerX)
    return r, theta

def compute_nLaps(theta):
    """
    Compute number of laps from angular values.

    Parameters : 
    theta : array-like
        Wrapped angles (radians).

    Returns : 
    nLaps : float
        Total laps = unwrapped_angle / (2π).
    """
    theta_unwrapped = np.unwrap(theta)
    delta_theta = theta_unwrapped[-1] - theta_unwrapped[0]
    nLaps = delta_theta / (2 * np.pi)
    return nLaps


for i, (t, x, y, inp) in enumerate(all_records):

    # STEP 1 — keep only samples AFTER first entry into target
    inside = (inp == 1)  # inp = colonne mouseInTarget
    first_inside_idx = np.where(inside)[0]

    if len(first_inside_idx) > 0:
        fi = first_inside_idx[0]
        t = t[fi:]
        x = x[fi:]
        y = y[fi:]
        inp = inp[fi:]
        inside = inside[fi:]

    # STEP 2 — compute radius + angle
    r, theta = compute_radius_angle(x, y, centerX, centerY)

    # STEP 3 — compute nLaps
    nLaps = compute_nLaps(theta)

    print(f"Record {i} : {nLaps:.2f} tours")

Record 0 : 4.65 tours
Record 1 : 5.05 tours
Record 2 : 4.04 tours
Record 3 : 4.34 tours
Record 4 : 5.54 tours


## Computation of Re (Effective Radius)

In [ ]:
# STEP 3 — Re = mean radius
for i, (t, x, y, inp) in enumerate(all_records):
    
    # STEP 1 — keep only samples AFTER first entry into target
    inside = (inp == 1)
    first_inside_idx = np.where(inside)[0]

    if len(first_inside_idx) > 0:
        fi = first_inside_idx[0]
        t = t[fi:]
        x = x[fi:]
        y = y[fi:]
        inp = inp[fi:]
        inside = inside[fi:]

    # STEP 2 — recalculate distance to center
    dist = np.sqrt((x - centerX)**2 + (y - centerY)**2)

    # STEP 3 — Re
    Re = np.mean(dist)

    print(f"Record {i} : Re = {Re:.2f} px")

Record 0 : Re = 249.99 px
Record 1 : Re = 242.84 px
Record 2 : Re = 245.70 px
Record 3 : Re = 241.88 px
Record 4 : Re = 239.13 px


## Computation of Te (Effective Tolerance)

In [ ]:

def compute_Te(x, y, centerX, centerY):
    """
    Compute effective tolerance Te from radial variability.

    Returns
    Te : float
        Effective tolerance = sigma * sqrt(2πe).
    sigma : float
        STD of radial distance.
    r : ndarray
        Radial distance time series.
    """

    # radius r(t)
    r = np.sqrt((x - centerX)**2 + (y - centerY)**2)

    # standard deviation of the radius
    sigma = np.std(r)

    # constant sqrt(2*pi*e)
    K = np.sqrt(2 * np.pi * np.e)

    # final Te
    Te = sigma * K

    return Te, sigma, r


# -------- VERSION WITH FIRST INSIDE FILTER ----------
for i, (t_rec, x_rec, y_rec, in_rec) in enumerate(all_records):

    # STEP 1 — keep only samples AFTER first entry into target
    inside = (in_rec == 1)
    first_inside_idx = np.where(inside)[0]

    if len(first_inside_idx) > 0:
        fi = first_inside_idx[0]
        t_rec = t_rec[fi:]
        x_rec = x_rec[fi:]
        y_rec = y_rec[fi:]
        in_rec = in_rec[fi:]
        inside = inside[fi:]

    # STEP 2 — TE calcul 
    Te, sigma, r = compute_Te(x_rec, y_rec, centerX, centerY)

    print(f"Record {i} : Te = {Te:.2f} px   (sigma = {sigma:.2f})")

Record 0 : Te = 51.56 px   (sigma = 12.48)
Record 1 : Te = 54.16 px   (sigma = 13.11)
Record 2 : Te = 42.57 px   (sigma = 10.30)
Record 3 : Te = 42.25 px   (sigma = 10.22)
Record 4 : Te = 54.40 px   (sigma = 13.16)


## Computation of IDe (Effective Index of Difficulty)

In [ ]:
def compute_IDe(Re, Te):
    """
    Compute effective index of difficulty.

    Returns
    IDe : float
        IDe = (2πRe) / Te.
    """
    return (2 * np.pi * Re) / Te


for i, (t_rec, x_rec, y_rec, in_rec) in enumerate(all_records):

    
    # STEP 1 — keep only data from first entry into the target
    
    inside = (in_rec == 1)
    first_inside_idx = np.where(inside)[0]

    if len(first_inside_idx) > 0:
        fi = first_inside_idx[0]
        t_rec = t_rec[fi:]
        x_rec = x_rec[fi:]
        y_rec = y_rec[fi:]
        in_rec = in_rec[fi:]
        inside = inside[fi:]

    
    # STEP 2 — compute Te using YOUR formula
    
    Te, sigma, r = compute_Te(x_rec, y_rec, centerX, centerY)

    Re = np.mean(r)

    
    # STEP 3 — compute IDe
    
    IDe = compute_IDe(Re, Te)

    print(f"Record {i} : IDe/Lap = {IDe:.2f}")

Record 0 : IDe/Lap = 30.47
Record 1 : IDe/Lap = 28.17
Record 2 : IDe/Lap = 36.26
Record 3 : IDe/Lap = 35.97
Record 4 : IDe/Lap = 27.62


## Computation of MT/lap (Movement Time Per Lap)

In [ ]:
# MT per lap

def compute_MT_per_lap(t, nLaps):
    """
    Compute movement time per lap.

    Returns: 
    MT_lap : float
        Duration / |nLaps|.
    """
    if len(t) < 2 or nLaps == 0:
        return np.nan
    duration = t[-1] - t[0]
    return duration / abs(nLaps)



# LOOP over all 5 records

for i, (t_rec, x_rec, y_rec, in_rec) in enumerate(all_records):

    
    # STEP 1 — keep only samples AFTER first entry into target
    
    inside = (in_rec == 1)
    first_inside_idx = np.where(inside)[0]

    if len(first_inside_idx) > 0:
        fi = first_inside_idx[0]
        t_f = t_rec[fi:]
        x_f = x_rec[fi:]
        y_f = y_rec[fi:]
        in_f = in_rec[fi:]
    else:
        t_f = t_rec
        x_f = x_rec
        y_f = y_rec
        in_f = in_rec

    
    # STEP 2 — compute theta and nLaps
    
    theta = np.arctan2(y_f - centerY, x_f - centerX)
    nLaps = compute_nLaps(theta)

    
    # STEP 3 — compute MT/lap
    
    MT_lap = compute_MT_per_lap(t_f, nLaps)

    print(f"Record {i+1} : MT/lap = {MT_lap:.3f} s/lap")

Record 1 : MT/lap = 4.292 s/lap
Record 2 : MT/lap = 3.807 s/lap
Record 3 : MT/lap = 4.693 s/lap
Record 4 : MT/lap = 4.480 s/lap
Record 5 : MT/lap = 3.476 s/lap


## Computation of IPe (Effective Performance Index)

In [ ]:
def compute_IPe(IDe, MT_lap):
    """
    Compute effective performance index.

    Returns

    IPe : float
        IPe = IDe / MT_lap.
    """
    return IDe / MT_lap


for i, (t_rec, x_rec, y_rec, in_rec) in enumerate(all_records):

    
    # STEP 1 — keep only samples AFTER first entry into target
   
    inside = (in_rec == 1)
    first_inside_idx = np.where(inside)[0]

    if len(first_inside_idx) > 0:
        fi = first_inside_idx[0]
        t_rec = t_rec[fi:]
        x_rec = x_rec[fi:]
        y_rec = y_rec[fi:]
        in_rec = in_rec[fi:]
        inside = inside[fi:]

    # STEP 2 — Re and Te 
  
    Te, sigma, r = compute_Te(x_rec, y_rec, centerX, centerY)
    Re = np.mean(r)


    # STEP 3 — nLaps with filtered data

    theta = np.arctan2(y_rec - centerY, x_rec - centerX)
    nLaps = compute_nLaps(theta)

    # STEP 4 — MT / lap
    
    MT_lap = compute_MT_per_lap(t_rec, nLaps)

  
    # STEP 5 — IDe
    
    IDe = compute_IDe(Re, Te)

    
    # STEP 6 — IPe
    IPe = compute_IPe(IDe, MT_lap)

    print(f"Record {i} : IPe = {IPe:.2f} bits/s")

Record 0 : IPe = 7.10 bits/s
Record 1 : IPe = 7.40 bits/s
Record 2 : IPe = 7.73 bits/s
Record 3 : IPe = 8.03 bits/s
Record 4 : IPe = 7.95 bits/s


## Computation of Error% (Angular Error Metric)

In [ ]:
def compute_error_angle(t_rec, x_rec, y_rec, in_rec, centerX, centerY):
    """
    Compute angular error as % of angle traveled outside target.

    Returns
    error : float
        (outside_angle / total_angle) * 100.
    """
    # STEP 1 — keep only samples AFTER first entry into target
    inside = (in_rec == 1)
    first_inside_idx = np.where(inside)[0]

    if len(first_inside_idx) > 0:
        fi = first_inside_idx[0]
        t_rec = t_rec[fi:]
        x_rec = x_rec[fi:]
        y_rec = y_rec[fi:]
        in_rec = in_rec[fi:]
        inside = inside[fi:]

    if len(t_rec) < 2:
        return np.nan  

    # STEP 2 — calculate error based on angular displacement
    # angle relative to center
    theta = np.arctan2(y_rec - centerY, x_rec - centerX)

    # increments of angle between samples
    dtheta = np.diff(theta)
    # unwrap to correct jumps at ±π
    dtheta = (dtheta + np.pi) % (2 * np.pi) - np.pi
    abs_dtheta = np.abs(dtheta)

    # segments outside the target (using the state of the starting point)
    is_outside = (in_rec[:-1] == 0)

    total_angle = np.sum(abs_dtheta)
    if total_angle <= 0:
        return 0.0

    outside_angle = np.sum(abs_dtheta[is_outside])

    error = outside_angle / total_angle * 100.0
    return error


# Loop over all records 
for i, (t_rec, x_rec, y_rec, in_rec) in enumerate(all_records):

    error = compute_error_angle(t_rec, x_rec, y_rec, in_rec, centerX, centerY)
    print(f"Record {i} : Error% = {error:.2f} %")

Record 0 : Error% = 5.94 %
Record 1 : Error% = 7.98 %
Record 2 : Error% = 0.78 %
Record 3 : Error% = 4.06 %
Record 4 : Error% = 15.68 %


## Computation of Be (Effective Bias Ratio)

In [ ]:
def compute_Be(ID_theory, IDe):
    """
    Compute effective bias ratio.

    Returns
    Be : float
        Ratio between theoretical difficulty and effective difficulty.
    """
    return ID_theory / IDe


for i, (t_rec, x_rec, y_rec, in_rec) in enumerate(all_records):

    
    # STEP 1 — keep only samples *after first entry into target*
    
    inside = (in_rec == 1)
    first_inside_idx = np.where(inside)[0]

    if len(first_inside_idx) > 0:
        fi = first_inside_idx[0]
        t_f = t_rec[fi:]
        x_f = x_rec[fi:]
        y_f = y_rec[fi:]
        in_f = in_rec[fi:]
    else:
        t_f = t_rec
        x_f = x_rec
        y_f = y_rec
        in_f = in_rec

    
    # STEP 2 — compute radius & angle AFTER filtering
   
    r, theta = compute_radius_angle(x_f, y_f, centerX, centerY)
    nLaps = compute_nLaps(theta)

    
    # STEP 3 — compute Te, Re (your Te_correct)
    
    Te, sigma, r_all = compute_Te(x_f, y_f, centerX, centerY)
    Re = np.mean(r_all)

    # STEP 4 — MT per lap
    
    MT_lap = compute_MT_per_lap(t_f, nLaps)

    
    # STEP 5 — compute IDe
   
    IDe = compute_IDe(Re, Te)

    
    # STEP 6 — compute Be
   
    Be = compute_Be(indexOfDifficulty, IDe)

    print(f"Record {i} : Be = {Be:.2f}")

Record 0 : Be = 1.09
Record 1 : Be = 1.17
Record 2 : Be = 0.91
Record 3 : Be = 0.92
Record 4 : Be = 1.20


## Final Summary Table (All Computed Metrics Per Record)


In [ ]:

# 1. Preparation of the final results table

results = []

# 2. Loop over the 5 records

for i, (t_rec, x_rec, y_rec, in_rec) in enumerate(all_records):

    # STEP 1 : filtering first entry 
    inside = (in_rec == 1)
    first_inside_idx = np.where(inside)[0]

    if len(first_inside_idx) > 0:
        fi = first_inside_idx[0]
        t_f = t_rec[fi:]
        x_f = x_rec[fi:]
        y_f = y_rec[fi:]
        in_f = in_rec[fi:]
    else:
        t_f = t_rec
        x_f = x_rec
        y_f = y_rec
        in_f = in_rec

    #  STEP 2 : Re, Te 
    Te, sigma, r = compute_Te(x_f, y_f, centerX, centerY) 
    Re = np.mean(r)

    #  STEP 3 : nLaps 
    theta = np.arctan2(y_f - centerY, x_f - centerX)
    nLaps = compute_nLaps(theta)

    # STEP 4 : MT 
    MT_lap = compute_MT_per_lap(t_f, nLaps)

    # STEP 5 : IDe 
    IDe = compute_IDe(Re, Te)

    #  STEP 6 : Be 
    Be = taskRadius / Te     

    # STEP 7 : IPe 
    IPe = compute_IPe(IDe, MT_lap)

    # STEP 8 : error 
    error = compute_error_angle(t_f, x_f, y_f, in_f, centerX, centerY)

    #  STOCKAGE 
    results.append({
        "Var": f"Rec{i+1:03d}",
        "nLaps": nLaps,
        "Re": Re,
        "Te": Te,
        "error": error,
        "MTlap": MT_lap,
        "IDelap": IDe,
        "Be": Be,
        "IPe": IPe
    })

In [11]:
print(f"{'Var':<7} | {'nLaps':>7} | {'Re':>8} | {'Te':>8} | {'error%':>8} | {'MT/lap':>8} | {'IDe/lap':>8} | {'Be':>6} | {'IPe':>8}")
print("-" * 95)

for r in results:
    print(f"{r['Var']:<7} | "
          f"{r['nLaps']:7.2f} | "
          f"{r['Re']:8.2f} | "
          f"{r['Te']:8.2f} | "
          f"{r['error']:8.2f} | "
          f"{r['MTlap']:8.2f} | "
          f"{r['IDelap']:8.2f} | "
          f"{r['Be']:6.2f} | "
          f"{r['IPe']:8.2f}")

Var     |   nLaps |       Re |       Te |   error% |   MT/lap |  IDe/lap |     Be |      IPe
-----------------------------------------------------------------------------------------------
Rec001  |    4.65 |   249.99 |    51.56 |     5.94 |     4.29 |    30.47 |   4.80 |     7.10
Rec002  |    5.05 |   242.84 |    54.16 |     7.98 |     3.81 |    28.17 |   4.57 |     7.40
Rec003  |    4.04 |   245.70 |    42.57 |     0.78 |     4.69 |    36.26 |   5.81 |     7.73
Rec004  |    4.34 |   241.88 |    42.25 |     4.06 |     4.48 |    35.97 |   5.86 |     8.03
Rec005  |    5.54 |   239.13 |    54.40 |    15.68 |     3.48 |    27.62 |   4.55 |     7.95
